In [2]:
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from model import *
from decode import *
from spell import *
from wer import *

In [ ]:
lipnet = LipNet(3,100,50,75,32,28)
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)

In [ ]:
spell = Spell(path='./grid.txt')
decoder = Decoder(greedy=False, beam_width=200, postprocessors=[labels_to_text, spell.sentence])

In [4]:
x = pd.read_csv('x_input.csv')
y = pd.read_csv('y_label.csv')
x = x.drop('Unnamed: 0',axis=1)
y = y.drop('Unnamed: 0',axis=1)
y['ctc_text'] += '____'
dataset = pd.merge(x, y)
dataset=dataset.sample(frac=1).reset_index(drop=True)

In [5]:
x_train = dataset.loc[:0.7*len(dataset)-1]
x_val = dataset.loc[0.7*len(dataset):0.9*len(dataset)-1]
x_test = dataset.loc[0.9*len(dataset):]
x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
assert len(dataset) == len(x_train) + len(x_val) + len(x_test)

In [11]:
tokens = {"_": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}
rev_tokens = {value : key for (key, value) in tokens.items()}
extra_tokens = { 'bos_token' :"<s>" , 'eos_token' :"</s>" , 'unk_token' : "<unk>" , 'pad_token' : "_" , 'word_delimiter_token' : "|" }
rev_extra_tokens = {value : key for (key, value) in extra_tokens.items()}

In [6]:
def generator(csv, start, batch_size = 16):
    inputs = []
    labels = []
    input_length = np.ones((batch_size,1))*75
    label_length = np.ones((batch_size,1))*32
    ids = []
    for i in range(min(batch_size,len(csv)-start)):
        ids.append(csv['ids'][start+i])
        x = np.load(csv['output_paths'][start+i])
        x = np.transpose(x , (0,2,1,3))
        y = []
        for c in csv['ctc_text'][start+i]:
            y.append(tokens[c])
        y = np.array(y)
        inputs.append(x)
        labels.append(y)
        
    return [np.array(inputs),np.array(labels),input_length,label_length] , np.zeros((batch_size,)) , ids

In [ ]:
def predict(array,ids,csv):
    
    y_pred = lipnet.model.predict(array)
    pred = decoder.decode(y_pred, [75])
    wers = []
    for i in range(len(pred)):
        predd = pred[i]
        wer = wer_sentence(pred.lower(), csv[ids]['transcriptions'])
        wers.append(wer)
    return wers

In [7]:
start_train = 1
batch_size_train = 1
start_val = 0
batch_size_val = 1
wer_train=[]
wer_val=[]

In [14]:
print('Epochs : ',epochs)
print('Start : ',start_train)
for i in range(100):
    
    xt,yt,it = generator(x_train, start_train, batch_size = batch_size_train)
    start_train += batch_size_train
    if start_train >= len(x_train):
        start_train = 0

    xv,yv,iv = generator(x_val, start_val, batch_size = batch_size_val)
    start_val += batch_size_val
    if start_val >= len(x_val):
        start_val = 0
        
    try:
        h = lipnet.model.fit(xt,yt,epochs = 1)
        epochs += 1
        wer_train += predict(xt[0],it,x_train)
        wer_val += predict(xv[0],iv,x_val)
    except:
        print('Error')
        continue
    print('Epochs : ',epochs)
    print('Start : ',start_train)

Epochs :  1
Start :  1
Epochs :  1
Start :  1
1/1 [==============================] - 8s 8s/step - loss: inf
Epochs :  2
Start :  2
Epochs :  2
Start :  2
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  3
Start :  3
Epochs :  3
Start :  3
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  4
Start :  4
Epochs :  4
Start :  4
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  5
Start :  5
Epochs :  5
Start :  5
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  6
Start :  6
Epochs :  6
Start :  6
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  7
Start :  7
Epochs :  7
Start :  7
1/1 [==============================] - 2s 2s/step - loss: inf
Epochs :  8
Start :  8
Epochs :  8
Start :  8
Error
Error
Epochs :  8
Start :  9
Epochs :  8
Start :  9
Error
Error
Epochs :  8
Start :  10
Epochs :  8
Start :  10
Error
Error
Epochs :  8
Start :  11
Epochs :  8
Start :  11
1/1 [===============

ValueError: Object arrays cannot be loaded when allow_pickle=False

ValueError: Object arrays cannot be loaded when allow_pickle=False